In [7]:
import pandas as pd
import sqlite3
import numpy as np
import warnings

warnings.filterwarnings('ignore')

print("--- [ขั้นตอนที่ 1 & 2] (ฉบับใหม่: กรองเฉพาะ 'หมวดหมู่อาหาร') ---")

db_filename = 'global_food_prices.db'
table_name = 'food_prices'

# 🔴 [แก้ไข] เอาคอมเมนต์ # ออกจาก query แล้ว 🔴
query = f"""
    SELECT 
        date, 
        country_code, 
        market, 
        commodity, 
        price_usd,
        category 
    FROM {table_name}
    WHERE 
        price_type = 'Retail' 
        AND category != 'non-food'
"""

print(f"กำลังเชื่อมต่อกับ '{db_filename}' และดึงข้อมูลเฉพาะ 'อาหาร'...")

try:
    conn = sqlite3.connect(db_filename)
    df = pd.read_sql(query, conn)
    conn.close()
    
    print(f"ดึงข้อมูลสำเร็จ! ได้ข้อมูลมา {len(df)} แถว")

    # --- 2. การเตรียมข้อมูล (เหมือนเดิม) ---
    rows_before = len(df)
    df.dropna(subset=['price_usd'], inplace=True)
    rows_after = len(df)
    print(f"[Pre-processing 1] ลบแถวที่ 'price_usd' เป็นค่าว่าง: ลบไป {rows_before - rows_after} แถว")

    df['date'] = pd.to_datetime(df['date'])
    print("[Pre-processing 2] แปลงคอลัมน์ 'date' เป็นชนิด datetime สำเร็จ")

    print("\n--- ข้อมูล 'หลัง' การทำความสะอาด (พร้อมใช้งาน) ---")
    print(df.info())
    
    print("\n--- ตรวจสอบสินค้า (อาหาร) ที่หลากหลายขึ้น ---")
    print(f"ตอนนี้ใน Data set มีสินค้าอาหารทั้งหมด {df['commodity'].nunique()} ชนิด")
    print("\nหมวดหมู่ที่ดึงมา (Categories):")
    print(df['category'].unique())


except Exception as e:
    print(f"!!! เกิดข้อผิดพลาด: {e}")

--- [ขั้นตอนที่ 1 & 2] (ฉบับใหม่: กรองเฉพาะ 'หมวดหมู่อาหาร') ---
กำลังเชื่อมต่อกับ 'global_food_prices.db' และดึงข้อมูลเฉพาะ 'อาหาร'...
ดึงข้อมูลสำเร็จ! ได้ข้อมูลมา 2648777 แถว
[Pre-processing 1] ลบแถวที่ 'price_usd' เป็นค่าว่าง: ลบไป 2912 แถว
[Pre-processing 2] แปลงคอลัมน์ 'date' เป็นชนิด datetime สำเร็จ

--- ข้อมูล 'หลัง' การทำความสะอาด (พร้อมใช้งาน) ---
<class 'pandas.core.frame.DataFrame'>
Index: 2645865 entries, 0 to 2648776
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   country_code  object        
 2   market        object        
 3   commodity     object        
 4   price_usd     float64       
 5   category      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 141.3+ MB
None

--- ตรวจสอบสินค้า (อาหาร) ที่หลากหลายขึ้น ---
ตอนนี้ใน Data set มีสินค้าอาหารทั้งหมด 681 ชนิด

หมวดหมู่ที่ดึงมา (Categories):
['cereals and tubers' 'meat, fish and eggs' 'oil and fats

In [ ]:
print("\n--- [ขั้นตอนที่ 3] เริ่มต้นการทำ Feature Engineering ---")

try:
    # 3.1 สร้าง Time-based Features (เหมือนเดิม)
    df['Year'] = df['date'].dt.year
    df['Month'] = df['date'].dt.month
    df['Quarter'] = df['date'].dt.quarter
    print("[Feature Eng. 1] สร้างตัวแปร Year, Month, Quarter สำเร็จ")

    # --- 3.2 สร้าง Categorical Features (One-Hot Encoding) ---
    
    # 🔴 [แก้ไข] เราจะเพิ่ม 'category' เข้าไปใน list นี้ด้วย 🔴
    features_to_encode = ['country_code', 'commodity', 'category'] 
    
    print(f"[Feature Eng. 2] กำลังแปลง {features_to_encode} ด้วย One-Hot Encoding...")
    
    df_encoded = pd.get_dummies(df, columns=features_to_encode, drop_first=True)
    print("[Feature Eng. 2] แปลง One-Hot Encoding สำเร็จ")

    # --- 3.3 เตรียมข้อมูล X และ y (เหมือนเดิม) ---
    
    y = df_encoded['price_usd']
    
    X = df_encoded.drop(columns=[
        'date',
        'market',
        'price_usd' 
        # (คอลัมน์ 'category' ที่เป็น text จะหายไปโดยอัตโนมัติ
        # เพราะถูกแทนที่ด้วย 'category_...' (0/1) ไปแล้ว)
    ])

    print("\n--- Feature Engineering เสร็จสิ้น! ---")
    print("โครงสร้าง Features (X) ทั้งหมด:")
    X.info() # (ตอนนี้คุณจะเห็นคอลัมน์ 'category_...' ใหม่ๆ เพิ่มเข้ามา)

except Exception as e:
    print(f"!!! เกิดข้อผิดพลาดระหว่างทำ Feature Engineering: {e}")


--- [ขั้นตอนที่ 3] เริ่มต้นการทำ Feature Engineering ---
[Feature Eng. 1] สร้างตัวแปร Year, Month, Quarter สำเร็จ
[Feature Eng. 2] กำลังแปลง ['country_code', 'commodity'] ด้วย One-Hot Encoding...
[Feature Eng. 2] แปลง One-Hot Encoding สำเร็จ

--- Feature Engineering เสร็จสิ้น! ---
ตัวอย่าง 'X' (Features ที่จะป้อนให้โมเดล):
             category  Year  Month  Quarter  country_code_AGO  \
0  cereals and tubers  2000      1        1             False   
1  cereals and tubers  2000      1        1             False   
2  cereals and tubers  2000      1        1             False   
3  cereals and tubers  2000      1        1             False   
4  cereals and tubers  2000      1        1             False   

   country_code_ARG  country_code_ARM  country_code_AZE  country_code_BDI  \
0             False             False             False             False   
1             False             False             False             False   
2             False             False             Fa

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_absolute_error

print("\n--- [ขั้นตอนที่ 4] เริ่มต้นการเปรียบเทียบโมเดล (แบบ Lab 3) ---")

try:
    # 4.1 แบ่งข้อมูล Train/Test (เราใช้ข้อมูลแค่ 2 แสนแถว จึงไม่ต้องกังวลเรื่องขนาด)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"แบ่งข้อมูล: {len(X_train)} แถวสำหรับ Train, {len(X_test)} แถวสำหรับ Test")

    # 4.2 สร้างรายชื่อโมเดลที่จะเปรียบเทียบ
    models = {
        "Linear Regression": LinearRegression(),
        "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
        "K-Neighbors Regressor": KNeighborsRegressor() 
        # เราจะไม่ใช้ Naive Bayes (จาก Lab 3) เพราะมันเหมาะกับ Classification มากกว่า
    }

    # 4.3 สร้าง List ว่างเพื่อเก็บผลลัพธ์
    model_names = []
    r2_scores = []
    mae_scores = []

    # 4.4 วนลูปเทรนและประเมินผลทีละโมเดล
    for name, model in models.items():
        print(f"\nกำลังเทรนโมเดล: {name}...")
        
        # เทรนโมเดล
        model.fit(X_train, y_train)
        
        # ทำนาย
        y_pred = model.predict(X_test)
        
        # วัดผล
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        
        # เก็บผลลัพธ์
        model_names.append(name)
        r2_scores.append(r2)
        mae_scores.append(mae)
        
        print(f"ผลลัพธ์ {name}: R-squared = {r2:.4f}, MAE = {mae:.4f}")

    # 4.5 สร้างตารางสรุปผล (Scoreboard)
    print("\n--- ตารางเปรียบเทียบประสิทธิภาพโมเดล (Scoreboard) ---")
    
    # สร้าง DataFrame เพื่อเปรียบเทียบ (เหมือนใน Lab 3)
    scoreboard = pd.DataFrame({
        'Algorithm': model_names,
        'R-squared': r2_scores,
        'Mean Absolute Error (MAE)': mae_scores
    })
    
    # เรียงลำดับตาม R-squared (คะแนนสูงสุดอยู่บน)
    scoreboard_sorted = scoreboard.sort_values(by='R-squared', ascending=False)
    
    print(scoreboard_sorted)

except Exception as e:
    print(f"!!! เกิดข้อผิดพลาดระหว่างการเทรนโมเดล: {e}")


--- [ขั้นตอนที่ 4] เริ่มต้นการเปรียบเทียบโมเดล (แบบ Lab 3) ---
แบ่งข้อมูล: 2116692 แถวสำหรับ Train, 529173 แถวสำหรับ Test

กำลังเทรนโมเดล: Linear Regression...
!!! เกิดข้อผิดพลาดระหว่างการเทรนโมเดล: could not convert string to float: 'miscellaneous food'


In [4]:
from sklearn.model_selection import GridSearchCV

print("\n--- [ขั้นตอนที่ 5] เริ่มต้นการปรับปรุงพารามิเตอร์ (Fine-tuning) ---")

try:
    # 5.1 กำหนด "Baseline" (ผลลัพธ์เดิมจากขั้นตอนที่ 4)
    baseline_r2 = 0.934104
    baseline_mae = 0.255472
    print(f"Baseline (Decision Tree): R2={baseline_r2:.4f}, MAE={baseline_mae:.4f}")

    # 5.2 กำหนดพารามิเตอร์ที่เราต้องการ "จูน"
    # เราจะลองหา 'max_depth' (ความลึกสูงสุดของต้นไม้)
    # และ 'min_samples_leaf' (จำนวนขั้นต่ำของข้อมูลที่ใบ)
    # เพื่อป้องกัน Overfitting และอาจทำให้โมเดลแม่นยำขึ้น
    
    param_grid = {
        'max_depth': [10, 20, 30, None], # ลองความลึกที่ 10, 20, 30, และไม่จำกัด (None)
        'min_samples_leaf': [1, 5, 10]   # ลองที่ 1, 5, และ 10
    }

    # 5.3 สร้างโมเดล GridSearchCV
    # cv=3 คือการแบ่งข้อมูล Train เพื่อ Cross-validate 3 รอบ
    # n_jobs=-1 คือการใช้ CPU ทุก Core เพื่อช่วยให้รันเร็วขึ้น
    
    # เราจะใช้ DecisionTreeRegressor ตัวเดิม
    dt_regressor = DecisionTreeRegressor(random_state=42)
    
    grid_search = GridSearchCV(
        estimator=dt_regressor, 
        param_grid=param_grid, 
        cv=3, 
        scoring='neg_mean_absolute_error', # เราจะจูนเพื่อลด MAE
        n_jobs=-1,
        verbose=1 # แสดงความคืบหน้า
    )

    print("\nกำลังเริ่ม GridSearchCV... (ขั้นตอนนี้อาจใช้เวลา 2-5 นาที)")
    
    # เริ่มการค้นหา! (ใช้ข้อมูล Train เท่านั้น)
    # (X_train และ y_train ยังอยู่ใน Memory จาก Cell ที่ 3)
    grid_search.fit(X_train, y_train)

    print("GridSearchCV เสร็จสิ้น!")
    
    # 5.4 แสดงผลลัพธ์ Parameters ที่ดีที่สุด
    print(f"\nParameters ที่ดีที่สุดที่ค้นพบ: {grid_search.best_params_}")

    # 5.5 ประเมินผลโมเดลที่ "จูนแล้ว"
    # ดึงโมเดลที่ดีที่สุดออกมา
    best_tuned_model = grid_search.best_estimator_
    
    # ทดสอบกับ Test Set
    y_pred_tuned = best_tuned_model.predict(X_test)
    
    # วัดผล
    tuned_r2 = r2_score(y_test, y_pred_tuned)
    tuned_mae = mean_absolute_error(y_test, y_pred_tuned)

    print("\n--- สรุปผลการ Fine-tuning ---")
    print(f"Baseline (โมเดลเดิม): R2={baseline_r2:.4f}, MAE={baseline_mae:.4f}")
    print(f"Tuned (โมเดลใหม่):   R2={tuned_r2:.4f}, MAE={tuned_mae:.4f}")

    # 5.6 คำนวณ % ที่ดีขึ้น
    # (เราจะเน้นที่ MAE เพราะเป็นสิ่งที่เราพยายามลด)
    mae_improvement_percent = ((baseline_mae - tuned_mae) / baseline_mae) * 100
    
    print("\n--- ผลลัพธ์สำหรับนำเสนอ (ข้อ 5) ---")
    if tuned_mae < baseline_mae:
        print(f"สรุป: เราปรับปรุงโมเดล 1 ตัว (Decision Tree) ได้สำเร็จ")
        print(f"ประสิทธิภาพดีขึ้น โดยลดค่า Error (MAE) ลงได้ {mae_improvement_percent:.2f} %")
    else:
        print("สรุป: โมเดลที่จูนแล้ว มีประสิทธิภาพใกล้เคียง/ด้อยกว่าโมเดลพื้นฐาน")
        print("แสดงว่าค่า Default ของ Decision Tree ทำงานได้ดีมากอยู่แล้ว")


except Exception as e:
    print(f"!!! เกิดข้อผิดพลาดระหว่างการทำ GridSearchCV: {e}")


--- [ขั้นตอนที่ 5] เริ่มต้นการปรับปรุงพารามิเตอร์ (Fine-tuning) ---
Baseline (Decision Tree): R2=0.9341, MAE=0.2555

กำลังเริ่ม GridSearchCV... (ขั้นตอนนี้อาจใช้เวลา 2-5 นาที)
!!! เกิดข้อผิดพลาดระหว่างการทำ GridSearchCV: name 'X_train' is not defined


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

print("\n--- [ขั้นตอนที่ 6] เริ่มต้นการวิเคราะห์และสร้างกราฟ (Visualization) ---")

sns.set_theme(style="whitegrid")

try:
    # --- กราฟที่ 1: แนวโน้มราคาระยะยาว (Long-term Trend) ---
    print("กำลังสร้าง กราฟที่ 1: แนวโน้มราคาระยะยาว (Line Plot)")
    
    df_yearly_price = df.groupby('Year')['price_usd'].mean().reset_index()
    
    # --- [แก้ไข] สร้าง Figure และ Axis (ax1) พร้อมกัน ---
    fig1, ax1 = plt.subplots(figsize=(12, 6))
    
    # --- [แก้ไข] สั่งให้ seaborn วาดลงบน ax1 ---
    sns.lineplot(data=df_yearly_price, x='Year', y='price_usd', marker='o', ax=ax1)
    
    ax1.set_title('Graph 1: Average Global Food Price Trend (USD) Over Years')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Average Price (USD)')
    plt.show()
    
    # --- กราฟที่ 2: เปรียบเทียบราคาสินค้าแต่ละประเภท (Commodity Comparison) ---
    print("\nกำลังสร้าง กราฟที่ 2: เปรียบเทียบราคาสินค้า (Bar Plot)")
    
    # --- [แก้ไข] สร้าง Figure และ Axis (ax2) พร้อมกัน ---
    fig2, ax2 = plt.subplots(figsize=(12, 6))
    
    # --- [แก้ไข] สั่งให้ seaborn วาดลงบน ax2 ---
    sns.barplot(data=df, x='commodity', y='price_usd', ax=ax2)
    
    ax2.set_title('Graph 2: Average Price (USD) by Commodity Type')
    ax2.set_xlabel('Commodity')
    ax2.set_ylabel('Average Price (USD)')
    plt.show()

    # --- กราฟที่ 3: การกระจายตัวของราคาตามฤดูกาล (Seasonality) ---
    print("\nกำลังสร้าง กราฟที่ 3: การกระจายตัวของราคา (Box Plot) [แบบซ่อน Outliers]")
    
    # --- [แก้ไข] สร้าง Figure และ Axis (ax3) พร้อมกัน ---
    fig3, ax3 = plt.subplots(figsize=(12, 6))
    
    # --- [แก้ไข] สั่งให้ seaborn วาดลงบน ax3 ---
    sns.boxplot(data=df, x='Month', y='price_usd', showfliers=False, ax=ax3) 
    
    ax3.set_title('Graph 3: Price Distribution by Month (Seasonality) - Outliers Hidden')
    ax3.set_xlabel('Month')
    ax3.set_ylabel('Price (USD)')
    plt.show()

except Exception as e:
    print(f"!!! เกิดข้อผิดพลาดระหว่างการสร้างกราฟ: {e}")
    print("!!! กรุณาตรวจสอบว่า DataFrame 'df' ยังอยู่ใน Memory")


--- [ขั้นตอนที่ 6] เริ่มต้นการวิเคราะห์และสร้างกราฟ (Visualization) ---
กำลังสร้าง กราฟที่ 1: แนวโน้มราคาระยะยาว (Line Plot)
!!! เกิดข้อผิดพลาดระหว่างการสร้างกราฟ: name 'df' is not defined
!!! กรุณาตรวจสอบว่า DataFrame 'df' ยังอยู่ใน Memory
